In [18]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

df_train = pd.read_csv('KDDTrain+.txt')
df_test = pd.read_csv('KDDTest+.txt')

columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
            'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
            'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
            'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count',
            'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
            'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
            'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
            'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
            'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'attack',
            'level']

df_train.columns = columns
df_test.columns = columns

df_train.drop('level', axis=1, inplace=True)
df_test.drop('level', axis=1, inplace=True)

df_train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal
1,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune
2,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal
3,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal
4,0,tcp,private,REJ,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune


Zbiór danych NSL-KDD to ulepszenie oryginalnego zbioru KDD Cup 99. KDD Cup 99 został stworzony w ramach konkursu Knowledge Discovery and Data Mining Cup 1999, zorganizowanego w celu zachęcenia do rozwoju technik detekcji ataków w systemach komputerowych.  
  
Oryginalny KDD Cup 99 zawierał dane zebrane z symulacji ruchu sieciowego w laboratoryjnym środowisku, a uczestnicy konkursu mieli za zadanie zidentyfikować ataki na sieci. Jednak później stwierdzono, że oryginalny zbiór danych ma pewne wady, takie jak nadmierna liczba powtarzających się rekordów.  
  
NSL-KDD powstał jako poprawiona wersja, jest bardziej zróżnicowany i realistyczny, a także bardziej odpowiedni do zastosowań w dzisiejszych systemach komputerowych. Zbiór ten składa się z różnych rodzajów ataków, takich jak ataki DoS (Denial of Service), probing, R2L (Remote-to-Local), U2R (User-to-Root) itp.  
  
Obecnie zbiór NSL-KDD jest używany jako benchmark w dziedzinie badawczej dotyczącej bezpieczeństwa sieci komputerowych i jest stosowany do testowania skuteczności różnych algorytmów i metod detekcji ataków. 

In [19]:
print(df_train.shape)
print(df_test.shape)
print(len(columns))

(125972, 42)
(22543, 42)
43


Cechy opisujące ruch sieciowy w zbiorze danych:
1. **duration:**  
Opisuje czas trwania połączenia w sekundach. Jest to liczba całkowita.

2. **protocol_type:**  
Określa typ protokołu komunikacyjnego używanego w danym połączeniu, np. "tcp", "udp", "icmp". Jest to atrybut kategoryczny.

3. **service:**  
Określa usługę, która jest używana w danym połączeniu, np. "http", "ftp", "telnet". Jest to atrybut kategoryczny.

4. **flag:**  
Oznacza flagę ustawioną w trakcie połączenia, np. "SF" (Established), "S0" (Connection attempt), "REJ" (Connection rejected).  
Jest to atrybut kategoryczny.

5. **src_bytes:**  
Ilość przesłanych bajtów z komputera źródłowego do docelowego w danym połączeniu. Jest to liczba całkowita.

6. **dst_bytes:**  
Ilość przesłanych bajtów z komputera docelowego do źródłowego w danym połączeniu. Jest to liczba całkowita.

7. **land:**  
Wartość binarna (0 lub 1) określająca, czy połączenie jest "land attack" (atak polegający na wysłaniu pakietu do ofiary   
z fałszywym adresem źródłowym równym adresem docelowym).

8. **wrong_fragment:**  
Liczba "złych" fragmentów w pakietach w danym połączeniu. Jest to liczba całkowita.

9. **urgent:**  
Liczba pakietów z flagą "urgent" (o wysokim priorytecie) w danym połączeniu. Jest to liczba całkowita.

10. **hot:**  
Określa liczbę "gorących" punktów (czyli często odwiedzanych) na serwerze. Jest to liczba całkowita.

11. **num_failed_logins:**  
Liczba nieudanych prób logowania przed danym połączeniem. Jest to liczba całkowita.

12. **logged_in:**  
Wartość binarna (0 lub 1) określająca, czy użytkownik jest zalogowany w danym połączeniu.

13. **num_compromised:**  
Liczba komputerów kompromitowanych w danym połączeniu. Jest to liczba całkowita.

14. **root_shell:**  
Wartość binarna (0 lub 1) określająca, czy w danym połączeniu został uzyskany dostęp do powłoki roota (administratora).

15. **su_attempted:**  
Wartość binarna (0 lub 1) określająca, czy w danym połączeniu była próba użycia polecenia su do uzyskania uprawnień roota.

16. **num_root:**  
Liczba uzyskanych dostępów roota w danym połączeniu. Jest to liczba całkowita.

17. **num_file_creations:**  
Liczba utworzonych plików w wyniku danego połączenia. Jest to liczba całkowita.

18. **num_shells:**  
Liczba powłok (shells), uruchomionych w wyniku danego połączenia. Jest to liczba całkowita.

19. **num_access_files:**  
Liczba dostępów do plików w wyniku danego połączenia. Jest to liczba całkowita.

20. **num_outbound_cmds:**  
Liczba wychodzących poleceń w danym połączeniu. W oryginalnym zbiorze KDD Cup '99 ta cecha zawsze wynosi 0, co oznacza,   
że nie jest informatywna.

21. **is_host_login:**  
Wartość binarna (0 lub 1) określająca, czy logowanie do hosta w danym połączeniu było wykonane jako część  
innego połączenia.

22. **is_guest_login:**  
Wartość binarna (0 lub 1) określająca, czy logowanie jako gość miało miejsce w danym połączeniu.

23. **count:**  
Liczba połączeń zidentyfikowanych w ciągu ostatnich 2 sekund przed danym połączeniem. Jest to liczba całkowita.

24. **srv_count:**  
Liczba połączeń do tego samego serwera (usługi) w ciągu ostatnich 2 sekund przed danym połączeniem.  
Jest to liczba całkowita.

25. **serror_rate:**  
Procent błędów związanych z połączeniem, zidentyfikowanych w ciągu ostatnich 2 sekund przed danym połączeniem.

26. **srv_serror_rate:**  
Procent błędów związanych z daną usługą, zidentyfikowanych w ciągu ostatnich 2 sekund przed danym połączeniem.

27. **rerror_rate:**  
Procent odrzuconych połączeń, zidentyfikowanych w ciągu ostatnich 2 sekund przed danym połączeniem.

28. **srv_rerror_rate:**  
Procent odrzuconych połączeń związanych z daną usługą, zidentyfikowanych w ciągu ostatnich 2 sekund przed   
danym połączeniem.

29. **same_srv_rate:**  
Procent połączeń do tego samego serwera (usługi) w ciągu ostatnich 2 sekund przed danym połączeniem.

30. **diff_srv_rate:**  
Procent połączeń do różnych serwerów (usług) w ciągu ostatnich 2 sekund przed danym połączeniem.

31. **srv_diff_host_rate:**  
Procent połączeń do różnych hostów w ciągu ostatnich 2 sekund przed danym połączeniem.

32. **dst_host_count:**  
Liczba połączeń do tego samego docelowego hosta w ciągu ostatnich 2 sekund przed danym połączeniem.  
Jest to liczba całkowita.

33. **dst_host_srv_count:**  
Liczba połączeń do tej samej usługi na docelowym hoście w ciągu ostatnich 2 sekund przed danym połączeniem.  
Jest to liczba całkowita.

34. **dst_host_same_srv_rate:**  
Procent połączeń do tej samej usługi na docelowym hoście w ciągu ostatnich 2 sekund przed danym połączeniem.

35. **dst_host_diff_srv_rate:**  
Procent połączeń do różnych usług na docelowym hoście w ciągu ostatnich 2 sekund przed danym połączeniem.

36. **dst_host_same_src_port_rate:**  
Procent połączeń do tego samego portu źródłowego na docelowym hoście w ciągu ostatnich 2 sekund przed  
danym połączeniem.

37. **dst_host_srv_diff_host_rate:**  
Procent połączeń do różnych hostów dla tej samej usługi na docelowym hoście w ciągu ostatnich 2 sekund  
przed danym połączeniem.

38. **dst_host_serror_rate:**  
Procent błędów związanych z połączeniem na docelowym hoście w ciągu ostatnich 2 sekund przed danym połączeniem.

39. **dst_host_srv_serror_rate:**  
Procent błędów związanych z daną usługą na docelowym hoście w ciągu ostatnich 2 sekund przed danym połączeniem.

40. **dst_host_rerror_rate:**  
Procent odrzuconych połączeń na docelowym hoście w ciągu ostatnich 2 sekund przed danym połączeniem.

41. **dst_host_srv_rerror_rate:**  
Procent odrzuconych połączeń związanych z daną usługą na docelowym hoście w ciągu ostatnich 2 sekund przed  
danym połączeniem.

42. **label:**  
Oznaczenie klasy, do której należy dane połączenie. Może to być etykieta opisująca rodzaj połączenia,  
np. "normal" (dla połączeń normalnych) lub etykieta opisująca rodzaj ataku.


#### Stworzenie etykiet mówiących czy dany rekord to atak DDoS czy też nie:

In [20]:
ddos_types = ['apache2','back','land','neptune','mailbomb','pod','processtable',
              'smurf', 'teardrop','udpstorm','worm']

is_ddos_train = []

for attack in df_train['attack']:
    if attack in ddos_types:
        is_ddos_train.append(1)
    else:
        is_ddos_train.append(0)

df_train['is_ddos'] = is_ddos_train

is_ddos_test = []

for attack in df_test['attack']:
    if attack in ddos_types:
        is_ddos_test.append(1)
    else:
        is_ddos_test.append(0)

df_test['is_ddos'] = is_ddos_test

df_train.drop('attack', axis=1, inplace=True)
df_test.drop('attack', axis=1, inplace=True)

df_train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,is_ddos
0,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,0
1,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,1
2,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,0
3,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0
4,0,tcp,private,REJ,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,1


#### Sprawdzenie czy w zbiorze danych występują puste komórki

In [21]:
max(df_train.isnull().sum())

0

In [22]:
max(df_test.isnull().sum())

0

#### Sprawdzenie zbalansowania zbioru:

In [23]:
df_train['is_ddos'].value_counts()

is_ddos
0    80045
1    45927
Name: count, dtype: int64

#### Zastosowanie metody one hot encoding na zmiennych kategorycznych:

In [24]:
protocol_encoding_train = pd.get_dummies(df_train['protocol_type'], dtype=int)
service_encoding_train = pd.get_dummies(df_train['service'], dtype=int)
flag_encoding_train = pd.get_dummies(df_train['flag'], dtype=int)

df_train = df_train.join(protocol_encoding_train)
df_train = df_train.join(service_encoding_train)
df_train = df_train.join(flag_encoding_train)


protocol_encoding_test = pd.get_dummies(df_test['protocol_type'], dtype=int)
service_encoding_test = pd.get_dummies(df_test['service'], dtype=int)
flag_encoding_test = pd.get_dummies(df_test['flag'], dtype=int)

df_test = df_test.join(protocol_encoding_test)
df_test = df_test.join(service_encoding_test)
df_test = df_test.join(flag_encoding_test)

df_train.drop(['protocol_type', 'service', 'flag'], inplace=True, axis=1)
df_test.drop(['protocol_type', 'service', 'flag'], inplace=True, axis=1)

is_ddos_column_train = df_train.pop('is_ddos')
df_train['is_ddos'] = is_ddos_column_train

is_ddos_column_test = df_test.pop('is_ddos')
df_test['is_ddos'] = is_ddos_column_test

df_train.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,RSTO,RSTOS0,RSTR,S0,S1,S2,S3,SF,SH,is_ddos
0,0,146,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,0,232,8153,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,0,199,420,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [25]:
print(df_train.shape)
print(df_test.shape)

(125972, 123)
(22543, 117)


#### Usunięcie ze zbioru treningowego kolumn, które nie występuje w zbiorze testowym:

In [26]:
columns_train = list(df_train.columns)
columns_test = list(df_test.columns)

for column in columns_train:
    if column not in columns_test:
        df_train.pop(column)

In [27]:
print(df_train.shape)
print(df_test.shape)

(125972, 117)
(22543, 117)


#### Standaryzacja danych:

In [28]:
scaler = StandardScaler()

Y_train = df_train['is_ddos']
df_train.drop('is_ddos', inplace=True, axis=1)
df_train_scaled = scaler.fit_transform(df_train)
X_train = df_train_scaled

Y_test = df_test['is_ddos']
df_test.drop('is_ddos', inplace=True, axis=1)
df_test_scaled = scaler.fit_transform(df_test)
X_test = df_test_scaled

In [29]:
print(X_train.shape)
print(X_test.shape)

(125972, 116)
(22543, 116)


#### Stworzenie architektury sieci neuronowej:

In [65]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(X_train.shape[1],))
model.add(tf.keras.layers.Dense(60, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(30, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 60)                7020      
                                                                 
 dropout_31 (Dropout)        (None, 60)                0         
                                                                 
 dense_45 (Dense)            (None, 30)                1830      
                                                                 
 dropout_32 (Dropout)        (None, 30)                0         
                                                                 
 dense_46 (Dense)            (None, 1)                 31        
                                                                 
Total params: 8881 (34.69 KB)
Trainable params: 8881 (34.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### Trenowanie modelu: 

In [66]:
history = model.fit(X_train, Y_train, epochs=10, batch_size=32)

Epoch 1/10
3937/3937 [==============================] - 10s 2ms/step - loss: 0.0433 - accuracy: 0.9873
Epoch 2/10
3937/3937 [==============================] - 9s 2ms/step - loss: 0.0108 - accuracy: 0.9978
Epoch 3/10
3937/3937 [==============================] - 9s 2ms/step - loss: 0.0082 - accuracy: 0.9982
Epoch 4/10
3937/3937 [==============================] - 9s 2ms/step - loss: 0.0064 - accuracy: 0.9985
Epoch 5/10
3937/3937 [==============================] - 9s 2ms/step - loss: 0.0054 - accuracy: 0.9985
Epoch 6/10
3937/3937 [==============================] - 9s 2ms/step - loss: 0.0057 - accuracy: 0.9986
Epoch 7/10
3937/3937 [==============================] - 9s 2ms/step - loss: 0.0050 - accuracy: 0.9988
Epoch 8/10
3937/3937 [==============================] - 9s 2ms/step - loss: 0.0044 - accuracy: 0.9988
Epoch 9/10
3937/3937 [==============================] - 8s 2ms/step - loss: 0.0044 - accuracy: 0.9988
Epoch 10/10
3937/3937 [==============================] - 9s 2ms/step - loss: 0.00

#### Ocena na zbiorze testowym:

In [67]:
model.evaluate(X_test, Y_test)

705/705 [==============================] - 1s 1ms/step - loss: 1.6396 - accuracy: 0.9458


[1.6395633220672607, 0.9458368420600891]

Nasz model uzyskał dokładność na poziomie 95% procent co oznacza, że z taką skutecznością będzie w stanie wykryć atak DDoS w konkretnej sieci.